In [1]:
import pandas as pd
import numpy as np

In [10]:


def single_game_player_df(week, year, day, file_name,
                         qb_file, rb_file, wr_file,
                         te_file, def_file,
                         team1,
                         team2):

    fanduel = pd.read_csv(file_name)
    fanduel = fanduel[fanduel['keep']=='x']
    fanduel = fanduel[(fanduel['Injury Indicator'] != 'IR') &
                  (fanduel['Injury Indicator'] != 'O') &
                  (fanduel['Injury Indicator'] != 'D')]
    fanduel2 = fanduel[['Player ID + Player Name', 'Nickname','Position', 'Team',
                    'Salary', 'Injury Indicator','FPPG']]
    fanduel2 = fanduel2[fanduel2['Position'] != 'D']
    fanduel2['player_wk_year'] = fanduel2['Nickname'] + " " + week + " " + year

    fanduel3 = fanduel[['Player ID + Player Name', 'Nickname','Position', 'Team',
                    'Salary', 'Injury Indicator','FPPG']]
    fanduel3 = fanduel3[fanduel3['Position'] == 'D']
    fanduel3['player_wk_year'] = fanduel3['Team'] + " " + week + " " + year

    fanduel4 = fanduel2.append(fanduel3)
    fanduel4 = fanduel4[['Player ID + Player Name', 'Position', 
                     'Team','Salary', 'Injury Indicator',
                    'player_wk_year','FPPG']]
    #fanduel4.sort_values(by = 'Salary', ascending = False)

    fanduel4['pos_rank'] = fanduel4.groupby(['Team','Position'])['Salary'].rank(ascending = False).astype(int)
    fanduel4['pos_rank'] = fanduel4['Position'] + fanduel4['pos_rank'].astype(str)

    fanduel5 = fanduel4[fanduel4['pos_rank'].isin(['QB1','RB1','RB2','WR1',
                                               'WR2','WR3','WR4','WR5',
                                              'TE1','TE2','D1','K1'])]

    qb_df = pd.read_csv(qb_file)
    qb_df['position'] = 'qb'
    rb_df = pd.read_csv(rb_file)
    rb_df['position'] = 'rb'
    wr_df = pd.read_csv(wr_file)
    wr_df['position'] = 'wr'
    te_df = pd.read_csv(te_file)
    te_df['position'] = 'te'
    def_df = pd.read_csv(def_file)
    def_df['position'] = 'def'
    def_df = def_df.rename(columns = {'team_wk_year':'player_wk_year'})

    df = qb_df.append(rb_df)
    df = df.append(wr_df)
    df = df.append(te_df)
    df = df.append(def_df)

    df = df.reset_index()
    df = df.drop(columns = ['Unnamed: 0', 'index'])

    df = df[(df['team_wk_year'].str.contains(team1)) |
            (df['team_wk_year'].str.contains(team2))]
    df_final = pd.merge(fanduel5, df, how = 'left', 
                    on = 'player_wk_year', suffixes = (None, None))

    #df_final['Prob2'] = df_final[['FPPG', 'Prob']].mean(skipna=True, axis = 1)
    df_final['Prob2'] = df_final['FPPG']

    df_final = df_final.drop_duplicates(subset = 'player_wk_year', keep = 'last')
    df_final = df_final.sort_values(by = 'Prob2', ascending = False)

    return df_final

In [11]:
df_final = single_game_player_df(week = '15',
                        year = '2022',
                        day = 'thu',
                        file_name = '2022week15_thu_night_fanduel_players.csv',
                        qb_file = 'week15_scored_qb.csv',
                        rb_file = 'week15_scored_rb.csv',
                        wr_file = 'week15_scored_wr.csv',
                        te_file = 'week15_scored_te.csv',
                        def_file = 'week15_scored_def.csv',
                        team1 = 'SF',
                        team2 = 'SEA')

In [12]:
df_final = df_final[['Player ID + Player Name','Position', 'Team', 'Salary',
                    'player_wk_year','Prob2']]#,'FPPG','Prob']]

In [13]:
df_final

,Player ID + Player Name,Position,Team,Salary,player_wk_year,Prob2
1,84665-55050:Christian McCaffrey,RB,SF,17500,Christian McCaffrey 15 2022,18.370000
3,84665-26483:Geno Smith,QB,SEA,14500,Geno Smith 15 2022,17.110000
4,84665-85798:Brandon Aiyuk,WR,SF,13000,Brandon Aiyuk 15 2022,13.220000
7,84665-27050:Tyler Lockett,WR,SEA,11500,Tyler Lockett 15 2022,13.170000
5,84665-103342:Kenneth Walker III,RB,SEA,12500,Kenneth Walker III 15 2022,12.110000
6,84665-73048:DK Metcalf,WR,SEA,12000,DK Metcalf 15 2022,11.900000
2,84665-85607:Brock Purdy,QB,SF,15000,Brock Purdy 15 2022,10.890000
8,84665-39793:George Kittle,TE,SF,10500,George Kittle 15 2022,10.030000
26,84665-12549:San Francisco 49ers,D,SF,10000,SF 15 2022,9.923077
11,84665-60745:Jason Myers,K,SEA,9000,Jason Myers 15 2022,9.384615


In [14]:
def create_single_game_simulations(df_final, team1, team2, 
                                   outcomes,
                                  players1, players2,
                                  players3, players4):
    print('team1:')
    print(team1)
    print('team2:')
    print(team2)
    result = pd.DataFrame() 
    team1_df = df_final[df_final['Team'] == team1]
    team1_df['pos_rank'] = team1_df.groupby('Position')['Salary'].rank(ascending = False).astype(int)
    team1_df['pos_rank'] = team1_df['Position'] + team1_df['pos_rank'].astype(str)
    team2_df = df_final[df_final['Team'] == team2]
    team2_df['pos_rank'] = team2_df.groupby('Position')['Salary'].rank(ascending = False).astype(int)
    team2_df['pos_rank'] = team2_df['Position'] + team2_df['pos_rank'].astype(str)
    
    
    #outcomes = ['HL',
    #            'HM',
    #            'HH',
    #            'ML',
    #            'MM',
    #            'MH'#,
                #'LL',
                #'LM',
                #'LH'
     #          ]
    
    for outcome in outcomes:
        print(outcome)
        if (outcome == 'HL'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2','D1','K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['K1'])]
        elif (outcome == 'HM'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2','D1','K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','TE1','K1'])]
        elif (outcome == 'HH'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2','D1','K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2','D1','K1'])]
        elif (outcome == 'ML'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','WR1','TE1','D1','K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['K1','D1'])]
        elif (outcome == 'MM'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','WR1','TE1','D1','K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','TE1','D1','K1'])]
        elif (outcome == 'MH'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['QB1','RB1','WR1','TE1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2'])]
        elif (outcome == 'LL'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['D1','K1','RB1','QB1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['D1','K1','RB1','QB1'])]
        elif (outcome == 'LM'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','WR1','TE1','D1'])]
        elif (outcome == 'LH'):
            team1_df2 = team1_df[team1_df['pos_rank'].isin(['K1'])]
            team2_df2 = team2_df[team2_df['pos_rank'].isin(['QB1','RB1','RB2','WR1','WR2','WR3',
                                                            'WR4','WR5','TE1','TE2','D1'])]
        
        team1_team2 = team1_df2.append(team2_df2)
        df_final_qb = team1_team2.sort_values('Prob2',ascending= False)
        df1 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
        df1 = df1.rename(columns = {'Player ID + Player Name':'flex1',
                                             'Salary':'flex1_sal','Prob2':'flex1_prob'})
        df2 = df_final_qb[['Player ID + Player Name', 'Team','Salary', 'Prob2']]
        df2 = df2.rename(columns = {'Player ID + Player Name':'flex2',
                                             'Salary':'flex2_sal','Prob2':'flex2_prob'})
        df3 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
        df3 = df3.rename(columns = {'Player ID + Player Name':'flex3',
                                            'Salary':'flex3_sal','Prob2':'flex3_prob'})
        df4 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
        df4 = df4.rename(columns = {'Player ID + Player Name':'flex4',
                                             'Salary':'flex4_sal','Prob2':'flex4_prob'})
        df5 = df_final_qb[['Player ID + Player Name','Team', 'Salary', 'Prob2']]
        df5 = df5.rename(columns = {'Player ID + Player Name':'flex5',
                                             'Salary':'flex5_sal','Prob2':'flex5_prob'})

        sim_df = pd.merge(df1, df2, how = 'cross')
        sim_df = pd.merge(sim_df, df3, how = 'cross')
        sim_df = pd.merge(sim_df, df4, how = 'cross')
        sim_df = pd.merge(sim_df, df5, how = 'cross')
        
        
        sim_df = sim_df[(sim_df['flex1'] != sim_df['flex2']) & (sim_df['flex1'] != sim_df['flex3']) &
                (sim_df['flex1'] != sim_df['flex4']) & (sim_df['flex1'] != sim_df['flex5']) &
                (sim_df['flex2'] != sim_df['flex3']) & (sim_df['flex2'] != sim_df['flex4']) &
                (sim_df['flex2'] != sim_df['flex5']) & (sim_df['flex3'] != sim_df['flex4']) &
                (sim_df['flex3'] != sim_df['flex5']) & (sim_df['flex4'] != sim_df['flex5'])]

        sim_df['total_salary'] = sim_df['flex1_sal'] + sim_df['flex2_sal'] + sim_df['flex3_sal'] + sim_df['flex4_sal'] + sim_df['flex5_sal']
        sim_df['total_proj'] = sim_df['flex1_prob'] + sim_df['flex2_prob'] + sim_df['flex3_prob'] + sim_df['flex4_prob'] + sim_df['flex5_prob']

        sim_df = sim_df[sim_df['total_salary'] <= 60000]
        sim_df = sim_df[sim_df['total_salary'] > 57000]
        if (len(sim_df) == 0):
            continue

        final= sim_df[['flex1', 'flex2', 'flex3', 'flex4', 'flex5','total_proj']]
        final['total_proj'] = final['total_proj'].astype(str)
        #final['total_salary'] = final['total_salary'].astype(str)
        print(len(final))
        n = len(final)
       # if (len(final) < 10000):
       #     n = len(final)
       # else:
       #     n = 10000
        final2 = pd.DataFrame()
        for i in range(0,n):
          #  print(i)
            l = final.iloc[i,:].values.tolist()
            #print(l)
            l.sort()
            if ((players1 in l) |
                (players2 in l) |
               (players3 in l) |
                (players4 in l)
               ):
            #print(l)
                temp = pd.DataFrame([l])
            
                final2 = final2.append(temp)
            
        final2 = final2.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')
        
        df_team_check = df_final_qb[['Player ID + Player Name','Team']]
        
        for i in range(1,6):
            print(i)
            final2 = pd.merge(final2, df_team_check, how = 'inner', left_on = i,
                          right_on = 'Player ID + Player Name',
                          suffixes = (None,None))
            team_col = 'Team' + str(i)
            final2[team_col] = final2['Team']
            final2 = final2.drop(columns = ['Team','Player ID + Player Name'])
        keep_list = []
        team1_v_team2 = []
        for i in range(0,len(final2)):
            team1_count = 0
            team2_count = 0
            #keep_list = []
            teams = final2[['Team1','Team2','Team3','Team4','Team5']]
            l = teams.iloc[i,:].values.tolist()
            #print(l)
            for j in l:
             #   print(j)
                
                if (j == team1):
                    team1_count = team1_count + 1
                elif (j == team2):
                    team2_count = team2_count + 1
             #   print(team1_count)
             #   print(team2_count)
            if ((team1_count > 4) | (team2_count > 4)):
                keep = 'No'
            else:
                keep = 'Yes'
                
            if ((team1_count == 2) & (team2_count == 3)):
                team1_team2 = '2:3'
            elif ((team1_count == 3) & (team2_count == 2)):
                team1_team2 = '3:2'
            elif ((team1_count == 4) & (team2_count == 1)):
                team1_team2 = '4:1'
            elif ((team1_count == 1) & (team2_count == 4)):
                team1_team2 = '1:4'
            else:
                team1_team2 = 'none'
                
          #  print(keep)
            keep_list.append(keep)
            team1_v_team2.append(team1_team2)
            #print(keep_list)
        final2['keep'] = keep_list
        final2['team1_team2_count'] = team1_v_team2
               
       # if(len(final2) < 33):
       #     m = len(final2)
       # else: 
       #     m = 33
       # final2 = final2.head(m)
        result = result.append(final2)
    
    result = result[result['keep'] == 'Yes']
    #result = result[[0,1,2,3,4,5]]
    result = result.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')

    
    return result

In [15]:
#fav: SF
#spread: -3
#o/u: 43.5


test = create_single_game_simulations(df_final = df_final,
                                      team1 = 'SF',
                                      team2 = 'SEA',
                                      outcomes = ['LL','LM','ML','MM','HL','LH'],
                                     players1 = '84665-55050:Christian McCaffrey',
                                     players2 = '84665-26483:Geno Smith',
                                     players3 = '84665-85607:Brock Purdy',
                                     players4 = '84665-103342:Kenneth Walker III')


team1:
SF
team2:
SEA
LL


<ipython-input-14-fa5088e668ca>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team1_df['pos_rank'] = team1_df.groupby('Position')['Salary'].rank(ascending = False).astype(int)
<ipython-input-14-fa5088e668ca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team1_df['pos_rank'] = team1_df['Position'] + team1_df['pos_rank'].astype(str)
<ipython-input-14-fa5088e668ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

1440
1
2
3
4
5
LM
ML


<ipython-input-14-fa5088e668ca>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['total_proj'] = final['total_proj'].astype(str)


1560
1
2
3
4
5
MM


...

19920
1
2
3
4
5
HL


<ipython-input-14-fa5088e668ca>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['total_proj'] = final['total_proj'].astype(str)


4680
1
2
3
4
5
LH


<ipython-input-14-fa5088e668ca>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['total_proj'] = final['total_proj'].astype(str)


840
1
2
3
4
5


In [16]:
test.shape

(181, 13)

In [17]:
test = test.drop_duplicates(subset = [1,2,3,4,5], keep = 'first')


In [18]:
test.sort_values(0, ascending = False).head(50)

,0,1,2,3,4,5,Team1,Team2,Team3,Team4,Team5,keep,team1_team2_count
0,62.29,84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-27050:Tyler Lockett,84665-6910:Robbie Gould,84665-73048:DK Metcalf,SEA,SEA,SEA,SF,SEA,Yes,1:4
50,62.183076923,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-39793:George Kittle,84665-73048:DK Metcalf,84665-85798:Brandon Aiyuk,SF,SEA,SF,SEA,SF,Yes,3:2
117,62.049230535,84665-26483:Geno Smith,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,84665-71705:Noah Fant,84665-73048:DK Metcalf,SEA,SF,SEA,SEA,SEA,Yes,1:4
29,61.854615385,84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-39793:George Kittle,84665-60745:Jason Myers,84665-85798:Brandon Aiyuk,SEA,SEA,SF,SEA,SF,Yes,2:3
13,61.817692308000005,84665-12550:Seattle Seahawks,84665-26483:Geno Smith,84665-39793:George Kittle,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,SEA,SEA,SF,SF,SEA,Yes,2:3
25,61.747692308,84665-103342:Kenneth Walker III,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-60745:Jason Myers,84665-85798:Brandon Aiyuk,SEA,SF,SEA,SEA,SF,Yes,2:3
0,61.710769231,84665-12549:San Francisco 49ers,84665-12550:Seattle Seahawks,84665-26483:Geno Smith,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,SF,SEA,SEA,SF,SEA,Yes,2:3
52,61.644615384999994,84665-26483:Geno Smith,84665-39793:George Kittle,84665-60745:Jason Myers,84665-73048:DK Metcalf,84665-85798:Brandon Aiyuk,SEA,SF,SEA,SEA,SF,Yes,2:3
53,61.537692308000004,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-60745:Jason Myers,84665-73048:DK Metcalf,84665-85798:Brandon Aiyuk,SF,SEA,SEA,SEA,SF,Yes,2:3
97,61.073076923,84665-103342:Kenneth Walker III,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-39793:George Kittle,84665-73048:DK Metcalf,SEA,SF,SEA,SF,SEA,Yes,2:3


In [19]:
#vegas odds:
#fav: SF
#spread: -3.5
#o/u: 43.5

In [20]:
#team1 = BUF, so as heavy fav we will subset only 4:1 and 3:2
team_ratio = ['4:1','3:2','2:3','1:4']

In [21]:
test = test.sort_values(0, ascending = False)#.head(150)
test2 = test[test['team1_team2_count'].isin(team_ratio)]

In [22]:
test2#.to_csv('thu_night_wk13_lineups_test.csv')

,0,1,2,3,4,5,Team1,Team2,Team3,Team4,Team5,keep,team1_team2_count
0,62.29,84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-27050:Tyler Lockett,84665-6910:Robbie Gould,84665-73048:DK Metcalf,SEA,SEA,SEA,SF,SEA,Yes,1:4
50,62.183076923,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-39793:George Kittle,84665-73048:DK Metcalf,84665-85798:Brandon Aiyuk,SF,SEA,SF,SEA,SF,Yes,3:2
117,62.049230535,84665-26483:Geno Smith,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,84665-71705:Noah Fant,84665-73048:DK Metcalf,SEA,SF,SEA,SEA,SEA,Yes,1:4
29,61.854615385,84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-39793:George Kittle,84665-60745:Jason Myers,84665-85798:Brandon Aiyuk,SEA,SEA,SF,SEA,SF,Yes,2:3
13,61.817692308000005,84665-12550:Seattle Seahawks,84665-26483:Geno Smith,84665-39793:George Kittle,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,SEA,SEA,SF,SF,SEA,Yes,2:3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,49.823076923,84665-103342:Kenneth Walker III,84665-12550:Seattle Seahawks,84665-6910:Robbie Gould,84665-73048:DK Metcalf,84665-85607:Brock Purdy,SEA,SEA,SF,SEA,SF,Yes,2:3
5,49.704615423,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,84665-6910:Robbie Gould,84665-85607:Brock Purdy,84665-86875:Jordan Mason,SF,SEA,SF,SF,SF,Yes,4:1
142,49.467692073,84665-12550:Seattle Seahawks,84665-55050:Christian McCaffrey,84665-6910:Robbie Gould,84665-71705:Noah Fant,84665-85607:Brock Purdy,SEA,SF,SF,SEA,SF,Yes,3:2
29,49.044615444,84665-55050:Christian McCaffrey,84665-60745:Jason Myers,84665-62055:Ray-Ray McCloud III,84665-6910:Robbie Gould,84665-85607:Brock Purdy,SF,SEA,SF,SF,SF,Yes,4:1


In [23]:
df_final

,Player ID + Player Name,Position,Team,Salary,player_wk_year,Prob2
1,84665-55050:Christian McCaffrey,RB,SF,17500,Christian McCaffrey 15 2022,18.370000
3,84665-26483:Geno Smith,QB,SEA,14500,Geno Smith 15 2022,17.110000
4,84665-85798:Brandon Aiyuk,WR,SF,13000,Brandon Aiyuk 15 2022,13.220000
7,84665-27050:Tyler Lockett,WR,SEA,11500,Tyler Lockett 15 2022,13.170000
5,84665-103342:Kenneth Walker III,RB,SEA,12500,Kenneth Walker III 15 2022,12.110000
6,84665-73048:DK Metcalf,WR,SEA,12000,DK Metcalf 15 2022,11.900000
2,84665-85607:Brock Purdy,QB,SF,15000,Brock Purdy 15 2022,10.890000
8,84665-39793:George Kittle,TE,SF,10500,George Kittle 15 2022,10.030000
26,84665-12549:San Francisco 49ers,D,SF,10000,SF 15 2022,9.923077
11,84665-60745:Jason Myers,K,SEA,9000,Jason Myers 15 2022,9.384615


In [24]:

def create_mvp_lineups(test_df,
                       df_final,
                       team_ratios,
                      counts,
                      mvps):

    test_df = pd.DataFrame()
    team1_team2 = team_ratios #['4:1','4:1','4:1' ,'4:1', '3:2','3:2','3:2','3:2']
    count = counts #[40,20,10,5,40,20,10,15]
    mvp = mvps #['84053-62239:Josh Allen','84053-25079:Stefon Diggs', '84053-12526:Buffalo Bills','84053-6642:Nick Folk',
      #'84053-62239:Josh Allen','84053-25079:Stefon Diggs',
      #'84053-111540:Rhamondre Stevenson','84053-90584:Mac Jones']
    n = len(team1_team2)

    for z in range(0,n):
    
        teams = team1_team2[z]
        counts = count[z]
        mvps_filter = mvp[z]
        print(teams)
        test_new = test2[test2['team1_team2_count'] == teams].head(counts)
    
        #x = 0
       # while(x < counts):
        for j in range(0,len(test_new)):
    
            df_final_mvp = df_final['Player ID + Player Name'][df_final['Player ID + Player Name'].isin([mvps_filter])].values.tolist()
            df_final_1 = df_final['Player ID + Player Name'].values.tolist()
            df_final_2 = df_final['Player ID + Player Name'].values.tolist()
            df_final_3 = df_final['Player ID + Player Name'].values.tolist()
            df_final_4 = df_final['Player ID + Player Name'].values.tolist()

            l = test_new.iloc[j, :].values.tolist()


            l_mvp = []
            l_1 = []
            l_2 = []
            l_3 = []
            l_4 = []

            for i in range(1,6):
               # print(l[i])
                if ((l[i] in df_final_mvp) & (len(l_mvp) < 1)):
                    l_mvp = l[i]
                    df_final_mvp.remove(l[i])
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])

                elif ((l[i] in (df_final_1)) & (len(l_1) < 1)):
                    l_1 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                    
                elif ((l[i] in (df_final_2)) & (len(l_2) < 1)):
                    l_2 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                elif ((l[i] in (df_final_3)) & (len(l_3) < 1)):
                    l_3 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
                elif ((l[i] in (df_final_4)) & (len(l_4) < 1)):
                    l_4 = l[i]
                    df_final_1.remove(l[i])
                    df_final_2.remove(l[i])
                    df_final_3.remove(l[i])
                    df_final_4.remove(l[i])
            
            final_l = [l_mvp, l_1, l_2, l_3, l_4]
            final_l = pd.DataFrame([final_l])

            #print(final_l)
            test_df = test_df.append(final_l)
            
    return test_df

In [25]:
team_ratios = ['4:1','4:1','4:1','4:1','4:1','4:1',
               
               '3:2','3:2','3:2','3:2','3:2','3:2',
               '3:2','3:2','3:2','3:2',
               
              '2:3','2:3','2:3','2:3','2:3','2:3',
               '2:3','2:3','2:3','2:3','2:3','2:3',
               
              '1:4','1:4','1:4','1:4',
               '1:4','1:4','1:4'
              ]
counts = [25,10,15,10,10,10,
          20,7,10,7,7,5,
          10,7,5,5,
          
         15,10,7,7,7,5,
         10,5,7,5,5,5,
          
         20,15,10,10,10,10,7
         ]
   
mvps = [
    '84665-55050:Christian McCaffrey','84665-85798:Brandon Aiyuk',
    '84665-85607:Brock Purdy','84665-39793:George Kittle',
    '84665-12549:San Francisco 49ers','84665-6910:Robbie Gould',
    
    '84665-55050:Christian McCaffrey','84665-85798:Brandon Aiyuk',
    '84665-85607:Brock Purdy','84665-39793:George Kittle',
    '84665-12549:San Francisco 49ers','84665-6910:Robbie Gould',
    '84665-26483:Geno Smith','84665-103342:Kenneth Walker III',
    '84665-27050:Tyler Lockett','84665-73048:DK Metcalf',
    
    '84665-26483:Geno Smith','84665-103342:Kenneth Walker III',
    '84665-27050:Tyler Lockett','84665-73048:DK Metcalf',
    '84665-12550:Seattle Seahawks','84665-60745:Jason Myers',
    '84665-55050:Christian McCaffrey','84665-85798:Brandon Aiyuk',
    '84665-85607:Brock Purdy','84665-39793:George Kittle',
    '84665-12549:San Francisco 49ers','84665-6910:Robbie Gould',    
    
    '84665-26483:Geno Smith','84665-103342:Kenneth Walker III',
    '84665-27050:Tyler Lockett','84665-73048:DK Metcalf',
    '84665-12550:Seattle Seahawks','84665-60745:Jason Myers',
    '84665-55050:Christian McCaffrey',
    
       ]


test_df = create_mvp_lineups(test_df = test2,
                       df_final = df_final,
                       team_ratios = team_ratios,
                      counts = counts,
                      mvps = mvps)

4:1
4:1
4:1
4:1
4:1
4:1
3:2
3:2
3:2
3:2
3:2
3:2
3:2
3:2
3:2
3:2
2:3
2:3
2:3
2:3
2:3
2:3
2:3
2:3
2:3
2:3
2:3
2:3
1:4
1:4
1:4
1:4
1:4
1:4
1:4


In [26]:
test_df.shape

(333, 5)

In [27]:
test_df

,0,1,2,3,4
0,84665-55050:Christian McCaffrey,84665-12549:San Francisco 49ers,84665-39793:George Kittle,84665-60745:Jason Myers,84665-85798:Brandon Aiyuk
0,84665-55050:Christian McCaffrey,84665-39793:George Kittle,84665-60745:Jason Myers,84665-6910:Robbie Gould,84665-85798:Brandon Aiyuk
0,84665-55050:Christian McCaffrey,84665-12549:San Francisco 49ers,84665-60745:Jason Myers,84665-6910:Robbie Gould,84665-85798:Brandon Aiyuk
0,84665-55050:Christian McCaffrey,84665-12549:San Francisco 49ers,84665-12550:Seattle Seahawks,84665-39793:George Kittle,84665-85798:Brandon Aiyuk
0,84665-55050:Christian McCaffrey,84665-103342:Kenneth Walker III,84665-12549:San Francisco 49ers,84665-39793:George Kittle,84665-6910:Robbie Gould
...,...,...,...,...,...
0,[],84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-39793:George Kittle,84665-60745:Jason Myers
0,[],84665-103342:Kenneth Walker III,84665-12549:San Francisco 49ers,84665-26483:Geno Smith,84665-60745:Jason Myers
0,84665-55050:Christian McCaffrey,84665-103342:Kenneth Walker III,84665-12550:Seattle Seahawks,84665-26483:Geno Smith,84665-71705:Noah Fant
0,[],84665-103342:Kenneth Walker III,84665-26483:Geno Smith,84665-71705:Noah Fant,84665-73048:DK Metcalf


In [28]:
if (len(test_df) < 150):
    n = len(test_df)
else:
    n = len(test_df)
test_df.head(n).to_csv('thu_night_wk15_lineups.csv')

In [150]:
test_df = pd.read_csv('thu_wk9_lineups.csv')
test_df

,MVP,1,2,3,4,5
0,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,NaN
1,82605-69531:Jalen Hurts,82605-27338:Rex Burkhead,82605-33047:Chris Moore,82605-71845:Miles Sanders,82605-72846:A.J. Brown,NaN
2,82605-69531:Jalen Hurts,82605-29932:Ka'imi Fairbairn,82605-69451:Jack Stoll,82605-71845:Miles Sanders,82605-72846:A.J. Brown,NaN
3,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-53390:Zach Pascal,82605-72846:A.J. Brown,NaN
4,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-72846:A.J. Brown,82605-90912:Kenneth Gainwell,NaN
...,...,...,...,...,...,...
145,82605-69531:Jalen Hurts,82605-29501:Brandin Cooks,82605-33047:Chris Moore,82605-53390:Zach Pascal,82605-72846:A.J. Brown,NaN
146,82605-69531:Jalen Hurts,82605-29501:Brandin Cooks,82605-33047:Chris Moore,82605-72846:A.J. Brown,82605-90912:Kenneth Gainwell,NaN
147,82605-69531:Jalen Hurts,82605-25813:Phillip Dorsett II,82605-33047:Chris Moore,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,NaN
148,82605-69531:Jalen Hurts,82605-60705:Tyron Johnson,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,82605-90576:DeVonta Smith,NaN


In [159]:
players = df_final['Player ID + Player Name'].values.tolist()
players

['82605-69531:Jalen Hurts',
 '82605-72846:A.J. Brown',
 '82605-86055:Dameon Pierce',
 '82605-71845:Miles Sanders',
 '82605-89628:Davis Mills',
 '82605-12545:Philadelphia Eagles',
 '82605-90576:DeVonta Smith',
 '82605-83074:Dallas Goedert',
 "82605-29932:Ka'imi Fairbairn",
 '82605-29501:Brandin Cooks',
 '82605-41235:Jake Elliott',
 '82605-12556:Houston Texans',
 '82605-27338:Rex Burkhead',
 '82605-33047:Chris Moore',
 '82605-39286:O.J. Howard',
 '82605-53390:Zach Pascal',
 '82605-90912:Kenneth Gainwell',
 '82605-25813:Phillip Dorsett II',
 '82605-73687:Quez Watkins',
 '82605-94094:Brevin Jordan',
 '82605-69451:Jack Stoll',
 '82605-64147:Britain Covey',
 '82605-60705:Tyron Johnson']

In [160]:
scores = [20.02, 13.9, 13.9, 15.3, 13.96, 8, 3.2, 20, 5,
         0, 3, 5, 2, 12.5, 1.9, 1.3, 11.3, 8.4, 3.5, 1,
         1.8, 0, 1.3]
#scores

In [161]:
v_df = pd.DataFrame({'players':players,
                     'scores':scores})
#v_df

In [162]:
test_df_v = pd.merge(test_df, v_df, how = 'inner', left_on = 'MVP', right_on = 'players' )
test_df_v['score_0'] = test_df_v['scores']*1.5
test_df_v = test_df_v.drop(columns = ['players', 'scores'])
test_df_v = pd.merge(test_df_v, v_df, how = 'inner', left_on = '1', right_on = 'players' )
test_df_v['score_1'] = test_df_v['scores']
test_df_v = test_df_v.drop(columns = ['players', 'scores'])
test_df_v = pd.merge(test_df_v, v_df, how = 'inner', left_on = '2', right_on = 'players' )
test_df_v['score_2'] = test_df_v['scores']
test_df_v = test_df_v.drop(columns = ['players', 'scores'])
test_df_v = pd.merge(test_df_v, v_df, how = 'inner', left_on = '3', right_on = 'players' )
test_df_v['score_3'] = test_df_v['scores']
test_df_v = test_df_v.drop(columns = ['players', 'scores'])
test_df_v = pd.merge(test_df_v, v_df, how = 'inner', left_on = '4', right_on = 'players' )
test_df_v['score_4'] = test_df_v['scores']
test_df_v = test_df_v.drop(columns = ['players', 'scores'])
test_df_v['total_score'] = test_df_v['score_0'] + test_df_v['score_1'] + test_df_v['score_2'] + test_df_v['score_3'] + test_df_v['score_4']
test_df_v = test_df_v.drop(columns = ['score_0','score_1','score_2','score_3','score_4'])
exp_score = test[0].head(150).values.tolist()
test_df_v['exp_score'] = exp_score
test_df_v['exp_score'] = test_df_v['exp_score'].astype(float)

In [163]:
test_df_v2 = test_df_v.head(150)

In [164]:
test_df_v2[test_df_v2['total_score'] >= 70]

,MVP,1,2,3,4,5,total_score,exp_score
0,82605-69531:Jalen Hurts,82605-12545:Philadelphia Eagles,82605-29932:Ka'imi Fairbairn,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,NaN,78.33,67.622856
1,82605-69531:Jalen Hurts,82605-29501:Brandin Cooks,82605-33047:Chris Moore,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,NaN,77.83,65.525714
2,82605-69531:Jalen Hurts,82605-29932:Ka'imi Fairbairn,82605-53390:Zach Pascal,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,NaN,71.63,65.403809
3,82605-69531:Jalen Hurts,82605-25813:Phillip Dorsett II,82605-33047:Chris Moore,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,NaN,84.83,65.157142
7,82605-69531:Jalen Hurts,82605-33047:Chris Moore,82605-53390:Zach Pascal,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,NaN,77.73,64.851427
10,82605-69531:Jalen Hurts,82605-25813:Phillip Dorsett II,82605-27338:Rex Burkhead,82605-72846:A.J. Brown,82605-83074:Dallas Goedert,NaN,74.33,64.323333
14,82605-69531:Jalen Hurts,82605-29932:Ka'imi Fairbairn,82605-72846:A.J. Brown,82605-73687:Quez Watkins,82605-83074:Dallas Goedert,NaN,72.43,63.737142
15,82605-69531:Jalen Hurts,82605-27338:Rex Burkhead,82605-33047:Chris Moore,82605-71845:Miles Sanders,82605-72846:A.J. Brown,NaN,73.73,63.594285
17,82605-69531:Jalen Hurts,82605-33047:Chris Moore,82605-39286:O.J. Howard,82605-71845:Miles Sanders,82605-72846:A.J. Brown,NaN,73.63,63.483333
50,82605-69531:Jalen Hurts,82605-33047:Chris Moore,82605-71845:Miles Sanders,82605-83074:Dallas Goedert,82605-90576:DeVonta Smith,NaN,81.03,61.439999


In [169]:
test_df_v.head(50).corr()

,5,total_score,exp_score
5,NaN,NaN,NaN
total_score,NaN,1.000000,0.704973
exp_score,NaN,0.704973,1.000000
